# ChiselDB：调试友好的结构化数据库

**ChiselDB：Debug-friendly structured database**

定位到错误现场后，我们还需要将它保存下来以进行分析，在传统上我们会使用波形图。但对于一些特定的信号（例如，总线上的消息），波形图并不是最好的选择。

Once we have located the error site, we need to save it for analysis. Traditionally, we would use waveforms. However, for certain specific signals (e.g., bus transactions), waveforms are not the best choice.

一方面，对于这些信号来讲，不是每个周期都有意义，我们只关心特定事件（例如，握手成功）发生时的信号值，用波形图记录所有周期的信息会产生大量存储浪费。

On the one hand, for these signals, not every cycle is meaningful. We only care about the signal values when specific events (e.g., handshake success) occur. Using waveforms to record information for all cycles results in significant storage waste.

另一方面，这些信号是结构化的（例如，Chisel Bundle），编译成 verilog，进而打印波形图时会丢失结构化信息，导致分析时需要人为关注信号之间的关系，增加了分析难度。

On the other hand, these signals are structured (e.g., Chisel Bundle). When compiled into Verilog and printed as waveforms, the structured information is lost, requiring manual attention to the relationships between signals during analysis, which increases the difficulty of analysis.

为了解决这些问题，我们提出了 ChiselDB。这是一个用于辅助功能和性能调试的结构化数据库。它使用 DPI-C 接口将 Chisel Bundle 的数据记录到 sqlite 数据库中。

To address these issues, we propose ChiselDB. ChiselDB is a structured database for aiding functional and performance debugging. It uses a DPI-C interface to log Chisel Bundle data into an SQLite database.

<div align="center">
  <img src="../images/02-functional/04-chiseldb/chiseldb-en.png" alt="chiselDB-overview" style="width: 30%; height: auto;">
</div>

稍后，我们可以用 SQL 查询语句对其进行高效分析。

Later, we can analyze it efficiently using SQL queries.

我们提供了一个预先编译的 `emu-cdb-err` 仿真程序，其存在一个人为注入的 bug，将从 L2 Cache 到 L3 Cache 释放数据都置为常量值。

We provide a prebuilt simulator `emu-cdb-err` with an injected bug that forces all data released from L2 Cache to L3 Cache to a constant value.

通过 `--dump-db` 参数启用 ChiselDB，同时启用 difftest，运行后 difftest 报错，并在 `./build` 路径下生成了 `.db` 文件。

Enable ChiselDB with `--dump-db` and turn on DiffTest; after running, DiffTest reports an error and a `.db` file is generated under `./build`.

In [ ]:
%%bash
source ../env.sh
cd ${NOOP_HOME}

rm -f ./build/*.db # clean old files
mkdir -p build

$(get_asset emu-precompile/emu-cdb-err) \
    -i $(get_asset workload/stream_100000.bin) \
    --diff $(get_asset emu-precompile/riscv64-nemu-interpreter-so) \
    --dump-db \
    2>linux.err || true

echo -n "Dump DB: "
realpath ./build/*.db

随后我们可以使用 sqlite 读取 db 文件并进行分析，下面查询所有地址为 `0x80048f00` 的 tilelink transaction，并使用 `./scripts/cache/convert_tllog.sh` 脚本来格式化输出。

Then use SQLite to read the `.db` for analysis: query all TileLink transactions at address `0x80048f00`, and format the output with `./scripts/cache/convert_tllog.sh`.

In [ ]:
%%bash
source ../env.sh

DB=$(ls -t ${NOOP_HOME}/build/*db | head -n 1)

sqlite3 ${DB} "select * from TLLog where ADDRESS=0x80048f00" | sh ${NOOP_HOME}/scripts/cache/convert_tllog.sh

Columns: timestamp to_from channel opcode permission address data

Here we can see that:

1. L1D writes data back to L2
```
16171 L2_L1D_0 C ProbeAckData Shrink TtoN 0 5 80048f00 0000000080048f50 0000000080014328 0000000000000000 0000000000000000 user: 0 echo: 0
16172 L2_L1D_0 C ProbeAckData Shrink TtoN 0 5 80048f00 0000000000000000 0000000000000000 000000008001e000 0000000080042060 user: 0 echo: 0
```
2. And to L3
```
16179 L3_L2_0 C ProbeAckData Shrink TtoN 0 2 80048f00 0000000080048f50 0000000080014328 0000000000000000 0000000000000000 user: 0 echo: 1
16180 L3_L2_0 C ProbeAckData Shrink TtoN 0 2 80048f00 0000000000000000 0000000000000000 000000008001e000 0000000080042060 user: 0 echo: 1
```
3. The next time L1D requests the data, L3->L2 GrantData does not match the original data written by L1D, indicating the bug.
```
16457 L2_L1D_0 A AcquireBlock Grow NtoT 0 0 80048f00 0000000000000000 0000000000000000 0000000000000000 0000000000000000 user: 80048f07 echo: 0 
16463 L3_L2_0 A AcquireBlock Grow NtoT 0 0 80048f00 0000000000000000 0000000000000000 0000000000000000 0000000000000000 user: 0 echo: 1 
16486 L3_L2_0 D GrantData Cap toT 1 0 80048f00 0000000000abcdef 0000000000000000 0000000000000000 0000000000000000 user: 0 echo: 1 
```